In [39]:
from bs4 import BeautifulSoup
import requests
import pdb
import pandas as pd
import re
from datetime import datetime
import json
import matplotlib.pyplot as plt
import urllib2
%matplotlib inline

## Scrape data from site

- Has to be AJAX post request, because the table is populated via JS and isn't fetched by BS
- Nonces change daily, so they must be fetched dynamically

In [2]:
class ScrapePepperData():
    """Dynamically scrape hot pepper table data from PepperScale Hot Pepper List"""
    def __init__(self):
        self.headers = {
            "user-agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36(KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36"
        }
        self.base_url = "https://www.pepperscale.com/hot-pepper-list/"
        self.ajax_url = "https://www.pepperscale.com/wp-admin/admin-ajax.php/"
        self.data = self.scrape_page()
        
    def fetch_nonces(self):
        """Nonces are dynamically generated on a regular (daily?) basis, and need to be scraped from the 
        <script> tags at the time of the scraping request"""
        request = urllib2.Request(self.base_url, headers=self.headers)
        page_html = urllib2.urlopen(request).read()
        return re.findall('"nonce":"(\w+)"', page_html)

    def launch_ajax_request(self, nonce):
        """Submit a POST request to AJAX URL to get JS-generated table content"""
        request_fields = {
            "start": 0,
            "length": 10000,
            "action":"gv_datatables_data",
            "view_id": 10294,
            "nonce": nonce,
         }
        
        return requests.post(self.ajax_url, headers=self.headers, 
                             data=request_fields, json={"key":"value"}).json()["data"]
    
    def scrape_page(self):
        """Try all nonces found on the page until the Golden Nonce is revealed!"""
        nonces = self.fetch_nonces()
        for nonce in nonces:
            try:
                return self.launch_ajax_request(nonce)
            except:
                pass

In [3]:
pepper_scraper = ScrapePepperData()
raw_pepper_data = pepper_scraper.data
len(raw_pepper_data)

122

In [36]:
INFO_LABELS = ["name", "link", "min_shu", "max_shu", "heat", "jrp", "species", "origin"]

def label_response_data(entry):
    labeled_entry = dict(zip(INFO_LABELS, entry))
    return labeled_entry

structured_pepper_data = [label_response_data(entry) for entry in raw_pepper_data]

## Verify base static data

Check that all 122 table entries made it into the raw data file

In [47]:
with open("../data/static_pepperscale_data.html", "r") as raw_html:
    pepper_html = BeautifulSoup(raw_html, 'html.parser')
    prettified_html = pepper_html.prettify()
    
with open("../data/static_prettified_pepperscale_data.html", "w") as f:
    f.write(prettified_html.encode('utf8'))

From spot check: tr class "even" and "odd" indicate top of row; there should be 122 entries, as there was on the site

In [48]:
len(pepper_html.find_all("tr", re.compile("even|odd")))

122

## Extract information

Process each pepper

Schema
- Name
- Heat (SHU)
- Jalapeño Reference Point
- Species
- Origin
- Link

Notes

- Can drop "child" tags with detailed information because info is in "tr even/odd" tags and child tags don't have min/max values
- Detailed information is unlabeled, but after min/max follows child tag order: "Heat", "JalRP", "Species", "Origin"
- Account for unicode characters in pepper names

In [49]:
INFO_LABELS = ["name", "min_shu", "max_shu", "heat", "jrp", "species", "origin", "link"]

In [50]:
pepper_tags = pepper_html.find_all("tr", re.compile("even|odd"))

In [51]:
test_row = pepper_tags[0]
test_row

<tr class="odd parent" role="row"><td style="" tabindex="0">Bell Pepper</td><td><a href="http://www.pepperscale.com/bell-pepper" rel="noopener noreferrer" target="_blank">pepperscale.com</a></td><td>0</td><td class="sorting_1">0</td><td style="display: none;">Mild</td><td style="display: none;">-8,000 to -2,500</td><td style="display: none;">annuum</td><td style="display: none;">Mexico and South America</td></tr>

In [77]:
def extract_info_from_row_tag(row_tag):
    """Extract pepper info from tag; ignore non-pepper-related info"""
    pepper_info = [element.text.encode("utf-8") for element in row_tag.contents if element.text != "pepperscale.com"]
#     link = row_tag.find("a").get('href')
    link = str(row_tag.find("a"))
    return dict(zip(INFO_LABELS, pepper_info + [link]))

all_peppers = [extract_info_from_row_tag(pepper_row) for pepper_row in pepper_tags]
all_peppers

[{'heat': 'Mild',
  'jrp': '-8,000 to -2,500',
  'link': '<a href="http://www.pepperscale.com/bell-pepper" rel="noopener noreferrer" target="_blank">pepperscale.com</a>',
  'max_shu': '0',
  'min_shu': '0',
  'name': 'Bell Pepper',
  'origin': 'Mexico and South America',
  'species': 'annuum'},
 {'heat': 'Mild',
  'jrp': '-8,000 to -2,500',
  'link': '<a href="http://www.pepperscale.com/gypsy-pepper" rel="noopener noreferrer" target="_blank">pepperscale.com</a>',
  'max_shu': '0',
  'min_shu': '0',
  'name': 'Gypsy Pepper',
  'origin': 'USA',
  'species': 'annuum'},
 {'heat': 'Mild',
  'jrp': '-8,000 to -2,500',
  'link': '<a href="http://www.pepperscale.com/purple-beauty-pepper" rel="noopener noreferrer" target="_blank">pepperscale.com</a>',
  'max_shu': '0',
  'min_shu': '0',
  'name': 'Purple Beauty Pepper',
  'origin': 'South America',
  'species': 'annuum'},
 {'heat': 'Mild',
  'jrp': '-8,000 to -2,500',
  'link': '<a href="http://www.pepperscale.com/melrose-pepper" rel="noopener 

## Validate & sanitize data

In [ ]:
peppers_data = pd.DataFrame(all_peppers)[INFO_LABELS]
peppers_data.head()

### Names

- Have to be unique

In [ ]:
peppers_data["name"].nunique() == len(peppers_data)

### Min/Max SHU (Scoville Heat Units)

In [ ]:
# min/max currently strings
peppers_data["min_shu"].values[0]

In [ ]:
(~peppers_data["min_shu"].str.isdigit()).sum()

In [ ]:
def sanitize_shu(shu):
    """astype(errors="ignore") throwing NA error"""
    try:
        return int(shu)
    except:
        return None

In [ ]:
peppers_data["min_shu"] = peppers_data["min_shu"].apply(sanitize_shu)
peppers_data["max_shu"] = peppers_data["max_shu"].apply(sanitize_shu)

In [ ]:
peppers_data["min_shu"].plot.hist(bins=20)

In [ ]:
peppers_data.boxplot("min_shu")

In [ ]:
# huge jump in SHU - definitely some major outliers in the scale here
peppers_data[peppers_data["min_shu"] > 500000].sort_values("min_shu").head(10)

In [ ]:
peppers_data["max_shu"].plot.hist(bins=20)

### JRP (Jalapeño reference point)

#### Fix malformed JRP ranges

In [ ]:
# malformed jrp ranges
peppers_data["jrp"].str.contains("to").sum(), len(peppers_data)

In [ ]:
malformed_jrp = peppers_data[~peppers_data["jrp"].str.contains("to")]
malformed_jrp

In [ ]:
def sanitize_jrp(jrp):
    jrp = jrp.replace("equal", "0")
    
    if jrp == "0":
        return [0,0]
    elif "to" not in jrp:
        return [int(val.replace(",", "")) for val in jrp.split(" - ")]
    return [int(val.replace(",", "")) for val in jrp.split(" to ")]


peppers_data[["min_jrp", "max_jrp"]] = pd.DataFrame(peppers_data["jrp"].apply(sanitize_jrp).values.tolist())
peppers_data.head()

### Categoricals

#### Heat

In [ ]:
peppers_data["heat"].value_counts(dropna=False)

In [ ]:
# record needs to be sanitized -- but what heat is it?

peppers_data[peppers_data["heat"] == ""]

In [ ]:
peppers_data[(peppers_data["min_shu"] >= 5000) & (peppers_data["max_shu"] <= 10000)]

In [ ]:
peppers_data["heat"] = peppers_data["heat"].apply(lambda x: x if x != "" else "Medium")
peppers_data["heat"].value_counts()

#### Species

In [ ]:
peppers_data["species"].value_counts()

In [ ]:
peppers_data["species"] = peppers_data["species"].apply(lambda x: x if x != "N/A" else None)
peppers_data["species"].value_counts(dropna=False)

#### Origin

In [ ]:
peppers_data["origin"].value_counts()

In [ ]:
def sanitize_origin(origin):
    if pd.isnull(origin):
        return origin
    elif "Mexico" in origin and "South America" in origin:
        return "Mexico, South America"
    elif origin in ["USA", "United States"]:
        return "United States"
    elif origin not in ["N/A", "Unknown"]:
        return origin
    
peppers_data["origin"] = peppers_data["origin"].apply(sanitize_origin)
peppers_data["origin"].value_counts(dropna=False)

##### Add region

Regions from here: https://www.thoughtco.com/official-listing-of-countries-world-region-1435153

In [ ]:
def add_region(origin):
    if origin in ["Italy", "United Kingdom", "Spain", "Hungary", "France"]:
        return "Europe"
    elif origin in ["United States", "Mexico"]:
        return "North America"
    elif origin in ["Trinidad", "Caribbean", "Jamaica",  "Panama", "Costa Rica"]:
        return "Central America and the Caribbean"
    elif origin in ["Peru", "Brazil", "Bolivia", "French Guyana", "South America"]:
        return "South America"
    elif origin in ["India", "Pakistan", "Thailand", "Japan", "China", "Phillipines"]:
        return "Asia"
    elif origin in ["Africa"]:
        return "Africa"
    elif origin in ["Australia"]:
        return "Australia and Oceania"
    elif origin in ["Syria"]:
        return "Middle East"
    elif pd.notnull(origin):
        return "Multi-Region"
    
peppers_data["region"] = peppers_data["origin"].apply(add_region)
peppers_data[["origin", "region"]].drop_duplicates()

## Convert to JSON

References:
- https://json-ld.org/spec/latest/json-ld-api-best-practices/
- https://www.sitepoint.com/google-maps-json-file/

In [ ]:
JSON_COLS = ["name", "species", "heat", "region", "origin", "min_shu", "max_shu", "min_jrp", "max_jrp", "link"]
json_peppers_data = peppers_data[JSON_COLS]
json_peppers_data.head()

In [ ]:
header_info = """{
    "source": "https://www.pepperscale.com/hot-pepper-list/",
    "contact": "https://github.com/alemosie",
    "last_updated": "%s",
    "peppers":
""" % (datetime.now())

with open ("data/peppers.json", "w") as json_file:
    json_file.write(header_info)
    json_file.write(json_peppers_data.to_json(orient='records'))
    json_file.write("}")

In [ ]:
str(datetime.now().date()).replace("-","")

## Class representation

In [40]:
class SanitizePepperData():
    """From nested object containing raw, solely parsed pepper data,
    create a clean dataset for analysis"""
    def __init__(self, pepper_data, write=False, output_path=None):
        self.raw = pd.DataFrame(pepper_data)

        clean_cols = ["name", "species", "heat", "region", "origin", "min_shu", "max_shu", "min_jrp", "max_jrp", "link"]
        self.clean = self.sanitize_pepper_data(self.raw.copy())[clean_cols]

        self.json = self.clean.to_dict(orient="records")
        if write:
            self.write_json(columns=clean_cols, output_path=output_path)

    #### GENERAL FUNCTIONS

    def sanitize_pepper_data(self, data):
        """Run all sanitization functions; produce clean copy of raw data"""
        data["min_shu"] = self.sanitize_shu("min_shu")
        data["max_shu"] = self.sanitize_shu("max_shu")
        data[["min_jrp", "max_jrp"]] = self.sanitize_jrp()
        data["heat"] = self.sanitize_heat()
        data["species"] = self.sanitize_species()
        data["origin"] = self.sanitize_field("origin", self.sanitize_origin_value)
        data["region"] = self.sanitize_field("origin", self.add_region_value)
        data["link"] = self.sanitize_link()
        return data

    def sanitize_field(self, field, value_sanitization_function):
        return self.raw[field].apply(value_sanitization_function)

    def write_json(self, output_path):
        header_info = """{
        "source": "https://www.pepperscale.com/hot-pepper-list/",
        "contact": "https://github.com/alemosie",
        "last_updated": "%s",
        "peppers":
        """ % (datetime.now())

        json_file = "data/peppers_{}.json".format(str(datetime.now().date()).replace("-","")) if not output_path else output_path
        print "Writing to %s..." % json_file
        with open (json_file, "w") as json_file:
            json_file.write(header_info)
            json_file.write(self.clean.to_json(orient='records'))
            json_file.write("}")

    #### FIELD FUNCTIONS

    ## Link

    def sanitize_link(self):
        return self.raw["link"].apply(lambda x: x.split('"')[1])

    ## Min/max SHU

    def sanitize_shu(self, shu_field):
        return self.raw[shu_field].apply(lambda x: int(x) if x != "" else None)

    ## JRP -> min/max JRP

    def sanitize_jrp_value(self, jrp):
        jrp = jrp.replace("equal", "0")

        if jrp == "0":
            return [0,0]
        elif "to" not in jrp:
            return [int(val.replace(",", "")) for val in jrp.split(" - ")]
        return [int(val.replace(",", "")) for val in jrp.split(" to ")]

    def sanitize_jrp(self):
        """creates dataframe with min and max JRP columns from raw JRP range"""
        return pd.DataFrame(self.raw["jrp"].apply(self.sanitize_jrp_value).values.tolist())

    ## Heat

    def sanitize_heat(self):
        return self.raw["heat"].apply(lambda x: x if x != "" else "Medium").str.lower()

    ## Species

    def sanitize_species(self):
        return self.raw["species"].apply(lambda x: x if x != "N/A" else None)

    ## Origin

    def sanitize_origin_value(self, origin):
        if pd.isnull(origin):
            return origin
        elif "Mexico" in origin and "South America" in origin:
            return "Mexico, South America"
        elif origin in ["USA", "United States"]:
            return "United States"
        elif origin not in ["N/A", "Unknown"]:
            return origin

    ## Region (new field to standardize origin)

    def add_region_value(self, origin):
        if origin in ["Italy", "United Kingdom", "Spain", "Hungary", "France"]:
            return "Europe"
        elif origin in ["United States", "Mexico"]:
            return "North America"
        elif origin in ["Trinidad", "Caribbean", "Jamaica",  "Panama", "Costa Rica"]:
            return "Central America and the Caribbean"
        elif origin in ["Peru", "Brazil", "Bolivia", "French Guyana", "South America"]:
            return "South America"
        elif origin in ["India", "Pakistan", "Thailand", "Japan", "China", "Phillipines"]:
            return "Asia"
        elif origin in ["Africa"]:
            return "Africa"
        elif origin in ["Australia"]:
            return "Australia and Oceania"
        elif origin in ["Syria"]:
            return "Middle East"
        elif pd.notnull(origin):
            return "Multi-Region"


In [78]:
SanitizePepperData(all_peppers).clean

,name,species,heat,region,origin,min_shu,max_shu,min_jrp,max_jrp,link
0,Bell Pepper,annuum,mild,Multi-Region,"Mexico, South America",0.0,0,-8000,-2500,http://www.pepperscale.com/bell-pepper
1,Gypsy Pepper,annuum,mild,Multi-Region,United States,0.0,0,-8000,-2500,http://www.pepperscale.com/gypsy-pepper
2,Purple Beauty Pepper,annuum,mild,South America,South America,0.0,0,-8000,-2500,http://www.pepperscale.com/purple-beauty-pepper
3,Melrose Pepper,annuum,mild,Multi-Region,United States,0.0,0,-8000,-2500,http://www.pepperscale.com/melrose-pepper
4,Peperone Di Senise,annuum,mild,Europe,Italy,0.0,0,-8000,2500,http://www.pepperscale.com/peperone-di-senise/
5,California Wonder Pepper,annuum,mild,North America,United States,0.0,0,-2500,-8000,https://www.pepperscale.com/california-wonder-...
6,Carmen Pepper,annuum,mild,Europe,Italy,0.0,0,-2500,-8000,https://www.pepperscale.com/carmen-pepper/
7,Tangerine Dream Pepper,annuum,mild,Multi-Region,United States,0.0,100,-8000,-250,https://www.pepperscale.com/tangerine-dream-pe...
8,Shishito Pepper,annuum,mild,Asia,Japan,50.0,200,-160,-13,http://www.pepperscale.com/shishito-pepper
9,Banana Pepper,annuum,mild,South America,South America,0.0,500,-8000,-5,http://www.pepperscale.com/banana-pepper
